ENVIRONMENT SETTING

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [1]:
import pandas as pd

In [3]:
import os
import time
import urllib
from tqdm import tqdm, trange

In [4]:
tqdm.pandas()

In [5]:
options = webdriver.ChromeOptions()
# options.add_argument('no-sandbox')
# options.add_argument('headless')
# options.add_argument('window-size=1920x1080')
# options.add_argument('disable-gpu')

In [6]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = options)

[WDM] - Downloading: 100%|██████████| 7.40M/7.40M [00:00<00:00, 25.6MB/s]


GET LINK

In [7]:
conference_dfs = []

In [8]:
for filename in tqdm(os.listdir('./data/conferences/')):
    conference_dfs.append(pd.read_csv(f'./data/conferences/{filename}', index_col = 0))

100%|██████████| 7/7 [00:00<00:00, 167.38it/s]


In [9]:
conference_df = pd.concat(conference_dfs).reset_index(drop = True)

In [10]:
conference_df

,title,link
0,Context-enriched molecule representations impr...,https://openreview.net/forum?id=kXXPLBEBVGH
1,The Slingshot Mechanism: An Empirical Study of...,https://openreview.net/forum?id=dJgYhYKvr1
2,Evident: a Development Methodology and a Knowl...,https://openreview.net/forum?id=_1bgdFHhA70
3,Efficient Sequence Packing without Cross-conta...,https://openreview.net/forum?id=e2M4CNa-UOS
4,Uniqueness and Complexity of Inverse MDP Models,https://openreview.net/forum?id=QYhUhMOI4C
...,...,...
9133,Transfer Learning for Estimating Causal Effect...,https://openreview.net/forum?id=ByzoVi0cFQ
9134,Generative Model For Material Irradiation Expe...,https://openreview.net/forum?id=Byg54oC5tQ
9135,Linearizing Visual Processes with Deep Generat...,https://openreview.net/forum?id=SkMPNoCcKQ
9136,NaN,https://openreview.net/forum?id=SkgD4jAcYX


CRAWL REVIEWS

In [11]:
progress = tqdm(total = len(conference_df))
dfs = []

  0%|          | 0/9138 [00:00<?, ?it/s]

In [ ]:
for title, link in conference_df.loc:
    try:
        driver.get(link)
        time.sleep(5)

        reviews = driver.find_elements(By.CSS_SELECTOR, 'div.note_with_children')

        for review in reviews:
            try:
                review_source = title
                review_link = link
                review_title = [x.text for x in review.find_elements(By.CSS_SELECTOR, 'div.title_pdf_row > h2.note_content_title > span')][0]
                review_signature = [x.text for x in review.find_elements(By.CSS_SELECTOR, 'span.signatures')][0]
                review_date = [x.text for x in review.find_elements(By.CSS_SELECTOR, 'span.date.item')][0]
                review_meta = [x.text for x in review.find_elements(By.CSS_SELECTOR, 'div.meta_row.pull-left > span.item:nth-child(2)')][0]
                review_subtitles = [x.text[:-1] for x in review.find_elements(By.CSS_SELECTOR, 'div.note_contents > span.note_content_field')][:len(review.find_elements(By.CSS_SELECTOR, 'div.note_contents > span.note_content_field')) - len(review.find_elements(By.CSS_SELECTOR, 'div.children div.note_contents')) + 1]
                review_contents = [x.text for x in review.find_elements(By.CSS_SELECTOR, 'div.note_contents > span.note_content_value')][:len(review.find_elements(By.CSS_SELECTOR, 'div.note_contents > span.note_content_field')) - len(review.find_elements(By.CSS_SELECTOR, 'div.children div.note_contents')) + 1]

                dfs.append(pd.DataFrame(data = {'source': review_source, 'link': review_link, 'title': review_title, 'signature': review_signature, 'date': review_date, 'meta': review_meta, 'subtitles': review_subtitles, 'contents': review_contents}))
            except:
                continue
    except:
        continue
    
    progress.update(1)

In [13]:
len(dfs)

102543

In [14]:
len(df)

328743

In [16]:
for i in range(int(len(df) / 50000) + 1):
    df.loc[50000 * i:50000 * (i + 1) - 1].reset_index(drop = True).to_csv(f'./data/papers/review{i + 1}.csv')

In [13]:
for i in range(int(len(df) / 50000) + 1):
    print(i)

0
1
2
3
4
5
6


In [17]:
import os

In [18]:
dffs = []

In [19]:
for filename in os.listdir('./data/papers'):
    dffs.append(pd.read_csv(f'./data/papers/{filename}', index_col = 0))

In [20]:
dff = pd.concat(dffs).reset_index()

In [23]:
dff.groupby('source').count()

,index,link,title,signature,date,meta,subtitles,contents
source,,,,,,,,
"""Hey, that's not an ODE'"": Faster ODE Adjoints with 12 Lines of Code",27,27,27,27,27,27,27,27
$$CONVOLUTION AND POOLING OPERATION MODULE WITH ADAPTIVE STRIDE PROCESSING EFFEC$$,34,34,34,34,34,34,34,33
$G^3$: Representation Learning and Generation for Geometric Graphs,51,51,51,51,51,51,51,51
$Q$-learning with regularization converges with non-linear non-stationary features,41,41,41,41,41,41,41,40
$\Delta$-PINNs: physics-informed neural networks on complex geometries,42,42,42,42,42,42,42,41
...,...,...,...,...,...,...,...,...
xTrimoABFold: Improving Antibody Structure Prediction without Multiple Sequence Alignments,45,45,45,45,45,45,45,44
{COMPANYNAME}11K: An Unsupervised Representation Learning Dataset for Arrhythmia Subtype Discovery,21,21,21,21,21,21,21,19
α\nVIL: Learning to Leverage Auxiliary Tasks for Multitask Learning,18,18,18,18,18,18,18,18
